<a href="https://colab.research.google.com/github/Praise-Atadja/CognitiveQuest/blob/main/Transfer_Learning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

def load_pretrained_model(model_name, input_shape, num_classes):
    """Load a pre-trained model with specified input shape and number of classes."""
    if model_name == 'ResNet50':
        base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'VGG16':
        base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'InceptionV3':
        base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError("Invalid model name. Please choose from 'ResNet50', 'VGG16', or 'InceptionV3'.")
    
    # Freeze all layers in the base model
    base_model.trainable = False
    
    # Add custom classification head
    x = GlobalAveragePooling2D()(base_model.output)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    # Combine base model and custom head
    model = Model(inputs=base_model.input, outputs=outputs)
    
    return model

def fine_tune_model(model, train_data, val_data, num_epochs=10, learning_rate=0.001):
    """Fine-tune the pre-trained model on the given dataset."""
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss=SparseCategoricalCrossentropy(),
                  metrics=[SparseCategoricalAccuracy()])
    
    # Train the model
    history = model.fit(train_data,
                        validation_data=val_data,
                        epochs=num_epochs)
    
    return history

def evaluate_model(model, test_data):
    """Evaluate the model on the test dataset."""
    loss, accuracy = model.evaluate(test_data)
    return loss, accuracy

def fine_tuning_analysis(model_name, train_data, val_data, test_data, input_shape, num_classes, num_epochs=10, learning_rate=0.001):
    """Perform fine-tuning analysis for a given pre-trained model."""
    # Load pre-trained model
    model = load_pretrained_model(model_name, input_shape, num_classes)
    
    # Fine-tune the model
    history = fine_tune_model(model, train_data, val_data, num_epochs, learning_rate)
    
    # Evaluate the model on test data
    test_loss, test_accuracy = evaluate_model(model, test_data)
    
    return history, test_loss, test_accuracy
